# 📊 Analisis Investasi Saham Bank

Notebook ini memungkinkan Anda:
- Memilih variabel prediktor
- Memilih bank
- Mengatur rasio kesehatan
- Melihat hasil prediksi harga saham
- Mendapatkan rekomendasi portofolio


In [ ]:
# Import modul
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from src.config.user_config import USER_CONFIG
from src.data_loader import load_data, filter_banks
from src.preprocessing import preprocess_data
from src.modeling import build_prediction_model
from src.analysis import analyze_ratio_impact
from src.optimization import optimize_portfolio
from src.visualization import plot_predictions


In [ ]:
# UI Konfigurasi
feature_selector = widgets.SelectMultiple(
    options=['ROA', 'ROE', 'NIM', 'NPL', 'CAR', 'EPS'],
    value=tuple(USER_CONFIG['PREDICTION_FEATURES']),
    description='Fitur:',
    disabled=False
)

bank_selector = widgets.SelectMultiple(
    options=USER_CONFIG['SELECTED_BANKS'],
    value=tuple(USER_CONFIG['SELECTED_BANKS']),
    description='Bank:',
    disabled=False
)

ratio_selector = widgets.SelectMultiple(
    options=['ROA', 'ROE', 'NIM', 'NPL', 'CAR', 'EPS'],
    value=tuple(USER_CONFIG['HEALTH_RATIOS']),
    description='Rasio:',
    disabled=False
)

max_bank_slider = widgets.IntSlider(
    value=USER_CONFIG['MAX_BANKS_IN_PORTFOLIO'],
    min=1, max=10, step=1,
    description='Jumlah Bank:'
)

run_button = widgets.Button(description="Jalankan Analisis")
output = widgets.Output()

def on_run_clicked(b):
    USER_CONFIG['PREDICTION_FEATURES'] = list(feature_selector.value)
    USER_CONFIG['SELECTED_BANKS'] = list(bank_selector.value)
    USER_CONFIG['HEALTH_RATIOS'] = list(ratio_selector.value)
    USER_CONFIG['MAX_BANKS_IN_PORTFOLIO'] = max_bank_slider.value
    
    with output:
        clear_output()
        print("🔄 Memuat data dan menjalankan analisis...")
        df = load_data("data/data_saham_bbca_bnga_2024.csv")
        if df is None:
            print("❌ Gagal memuat data.")
            return
        df = preprocess_data(df)
        df = filter_banks(df)
        model, df = build_prediction_model(df)
        if model:
            plot_predictions(df)
            analyze_ratio_impact(df)
            optimize_portfolio(df)
        else:
            print("❌ Model tidak berhasil dibuat.")

run_button.on_click(on_run_clicked)

display(widgets.VBox([
    widgets.HBox([feature_selector, bank_selector]),
    widgets.HBox([ratio_selector, max_bank_slider]),
    run_button,
    output
]))